In [ ]:
import os
import geopandas as gpd
from rasterio.mask import mask
import rasterio

def clip_raster_with_shapefile(raster_path, shapefile_path, output_folder):
    # Extract the subfolder and filename from the raster_path
    raster_subfolder = os.path.basename(os.path.dirname(raster_path))
    raster_filename = os.path.splitext(os.path.basename(raster_path))[0]

    # Extract the subfolders from the shapefile_path
    shapefile_subfolders = os.path.relpath(os.path.dirname(shapefile_path), r"plotType_Tower").split(os.path.sep)

    # Create the output subfolder and filename
    subfolder = os.path.join(output_folder, raster_subfolder, *shapefile_subfolders)
    os.makedirs(subfolder, exist_ok=True)
    
    filename = f"{os.path.splitext(os.path.basename(shapefile_path))[0]}_dtm.tif"
    output_path = os.path.normpath(os.path.join(subfolder, filename))

    # Read raster
    with rasterio.open(raster_path) as src:
        # Read shapefile
        gdf = gpd.read_file(shapefile_path)

        # Make sure the coordinate systems match
        assert gdf.crs == src.crs

        # Clip raster using geopandas geometry
        out_image, out_transform = mask(src, gdf.geometry, crop=True, invert=False)

        # Update metadata to match the clipped portion
        out_meta = src.meta.copy()
        out_meta.update({
            "driver": "GTiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            "transform": out_transform,
            "crs": src.crs
        })

        # Write the clipped raster to a new file
        with rasterio.open(output_path, "w", **out_meta) as dest:
            dest.write(out_image)

    print(f"Clipped raster saved to: {output_path}")

# Define paths
raster_folder = r"dtm_mosaicked"
shapefile_folder = r"plotType_Tower"
output_folder = r"plotType_Tower"

# Iterate through raster files and shapefiles to clip the raster
for raster_root, raster_dirs, raster_files in os.walk(raster_folder):
    for raster_file in raster_files:
        if raster_file.endswith(".tif"):
            raster_path = os.path.join(raster_root, raster_file)
            # Extract subfolder name from raster path
            raster_subfolder = os.path.basename(os.path.dirname(raster_path))
            # Construct the path for the corresponding shapefile subfolder
            shapefile_subfolder = raster_subfolder
            shapefile_subfolder_path = os.path.join(shapefile_folder, shapefile_subfolder)
            # Iterate through shapefiles in the corresponding subfolder
            for shapefile_root, shapefile_dirs, shapefile_files in os.walk(shapefile_subfolder_path):
                for shapefile_file in shapefile_files:
                    if shapefile_file.endswith(".shp"):
                        shapefile_path = os.path.join(shapefile_root, shapefile_file)
                        clip_raster_with_shapefile(raster_path, shapefile_path, output_folder)

print("Processing complete.")